In [1]:
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import numpy as np
from datetime import datetime
    
TRAIN_PATH = '../../res/ftr/base_data_train.csv'
EVALUATION_PATH = '../../res/ftr/base_data_evaluation.csv'

pd.set_option('display.max_columns', 30)

training_set = pd.read_csv(TRAIN_PATH)
evaluation_set = pd.read_csv(EVALUATION_PATH)

training_set.drop(columns=['Unnamed: 0'], inplace=True)
evaluation_set.drop(columns=['Unnamed: 0'], inplace=True)

In [2]:
def load_features(train_df, features_list):
    for feature in features_list:
        test_ftr = pd.read_csv('../../res/ftr/'+feature+'_train.csv')
    
        train_df = train_df.merge(test_ftr, on='id', how='inner').drop(columns=['Unnamed: 0'])
    
    return train_df
    
features_array = ['amenities', 'avenida_in_direction', 'encoded_provincia', 'encoded_tipodepropiedad', 'feature_hashed_ciudad',
                  'mean_precio_encoded_ciudad', 'mean_precio_encoded_provincia', 'mean_precio_encoded_tipodepropiedad',
                  'metros_totales_y_cubiertos_log', 'murder_rate_of_entity','provincia_borders_analysis', 'provincia_economy',
                  'qualificative_adjectives_in_description', 'surface_features', 'mean_idzona_price', 'dolar_for_date']
dataset = load_features(training_set, features_array)

In [ ]:
dataset_X = dataset.drop(columns=['precio']).values
dataset_Y = dataset['precio'].values

lgb_dataset = lgb.Dataset(dataset_X, dataset_Y)

params = {'objective': 'regression'}

cv_results = lgb.cv(params, lgb_dataset, num_boost_round = 100, nfold = 10, metrics = 'mae', early_stopping_rounds = 10, stratified = False)

In [ ]:
print("Final mean rmse score: %i" % (cv_results['l1-mean'][-1]))